In [127]:
import pandas as pd
import random
import os
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn import  metrics
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score

from sklearn import preprocessing
from sklearn.ensemble import RandomForestClassifier

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

In [128]:
class CFG:
    SEED = 42

In [129]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
seed_everything(CFG.SEED) # Seed 고정

In [130]:
train = pd.read_csv('./train.csv')  # 262
test = pd.read_csv('./test.csv')


train_X = train[:210]
test_X = train[210:]

def get_x_y(df):
    if 'class' in df.columns:
        df_x = df.drop(columns=['id', 'class'])
        df_y = df['class']
        return df_x, df_y
    else:
        df_x = df.drop(columns=['id'])
        return df_x

train_x, train_y = get_x_y(train_X)
test_x, test_y = get_x_y(test_X)

def cross_validation(classifier, features, labels):
    cv_scores = []
    
    for i in range(10):
        scores = cross_val_score(classifier, features, labels, cv=10, scoring = 'accuracy')
        cv_scores.append(scores.mean())
        
    
    return cv_scores

train_x

,father,mother,gender,trait,SNP_01,SNP_02,SNP_03,SNP_04,SNP_05,SNP_06,SNP_07,SNP_08,SNP_09,SNP_10,SNP_11,SNP_12,SNP_13,SNP_14,SNP_15
0,0,0,0,2,G G,A G,A A,G A,C A,A A,A A,G G,A A,G G,A G,A A,A A,A A,A A
1,0,0,0,2,A G,A G,C A,A A,A A,A G,A A,G A,A A,A G,A A,G A,G G,A A,A A
2,0,0,0,2,G G,G G,A A,G A,C C,G G,A A,G A,G A,A G,A A,A A,A A,A A,A A
3,0,0,0,1,A A,G G,A A,G A,A A,G G,G G,A A,G G,A G,G G,G G,G G,A A,G G
4,0,0,0,2,G G,G G,C C,A A,C C,A A,A A,A A,A A,G G,A A,A A,A G,A A,G A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
205,0,0,0,2,A G,A G,C A,A A,C A,A G,A A,G A,A A,G G,G G,G A,G G,A A,G A
206,0,0,0,2,G G,A G,C A,A A,A A,A A,G A,G G,A A,G G,A A,A A,A G,A A,A A
207,0,0,0,2,G G,A G,C A,G G,C A,G G,A A,G A,G A,G G,A G,A A,A G,A A,A A
208,0,0,0,2,G G,A A,C A,A A,A A,A G,A A,G G,A A,G G,A A,A A,A G,C A,G G


In [131]:
class_le = preprocessing.LabelEncoder()
snp_le = preprocessing.LabelEncoder()
snp_col = [f'SNP_{str(x).zfill(2)}' for x in range(1,16)]

snp_data = []
for col in snp_col:
    snp_data += list(train_x[col].values)

train_y = class_le.fit_transform(train_y)
snp_le.fit(snp_data)


LabelEncoder()

In [132]:
for col in train_x.columns:
    if col in snp_col:
        train_x[col] = snp_le.transform(train_x[col])
        test_x[col] = snp_le.transform(test_x[col])

In [133]:
cv = CountVectorizer()
x_traincv = cv.fit_transform(train_x)


mnb = MultinomialNB()
y_train = train_y.astype('int')
mnb.fit(x_traincv, y_train)

ValueError: Found input variables with inconsistent numbers of samples: [19, 210]

In [125]:
preds = clf.predict(test_x)
print('Done.')


test_x['class']=class_le.inverse_transform(preds)
pred = test_x['class']

print(pred)
print(test_y)
accuracy_score(test_y, pred)

Done.
210    C
211    B
212    A
213    A
214    B
215    C
216    A
217    A
218    C
219    B
220    A
221    B
222    B
223    B
224    A
225    B
226    B
227    B
228    B
229    A
230    C
231    C
232    B
233    C
234    B
235    C
236    B
237    A
238    C
239    A
240    A
241    C
242    C
243    B
244    B
245    B
246    A
247    B
248    A
249    B
250    A
251    C
252    B
253    A
254    B
255    B
256    B
257    B
258    C
259    A
260    A
261    B
Name: class, dtype: object
210    C
211    B
212    A
213    A
214    B
215    C
216    A
217    A
218    C
219    B
220    A
221    B
222    B
223    B
224    A
225    B
226    B
227    B
228    B
229    A
230    C
231    C
232    B
233    C
234    B
235    C
236    B
237    A
238    C
239    A
240    A
241    C
242    C
243    B
244    B
245    B
246    A
247    B
248    A
249    B
250    A
251    C
252    B
253    A
254    B
255    B
256    B
257    B
258    C
259    A
260    A
261    B
Name: class, dtype: object


1.0

In [18]:
submit = pd.read_csv('./sample_submission.csv')

submit['class'] = class_le.inverse_transform(preds)

submit.to_csv('./answer/submit.csv', index=False)

In [ ]:
train_x, train_y = get_x_y(train_X)
test_x, test_y = get_x_y(test_X)